In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import FVMtools as fvm
import time

In [ ]:
def adsmodel(t, cq, h, Dax, v, ec, ee, kads, kdes, qmax, cin):
    N = int(len(cq)/2)
    c = cq[:N]
    q = cq[N:]
    # adsorption
    rads = kads*c*(qmax-q) - kdes*q
    # discr
    [A2,A2f] = fvm.FVMdisc2nd(N, h, '3pc')
    [A1,A1f] = fvm.FVMdisc1st(N, h, '2pb')
    [B1,B0] = fvm.FVMdiscBV(N, h, [0, 1], [[1, -cin],[0, 0]])
    # assembly
    dcdt = Dax*(A2@c + A2f@(B1@c + B0)) \
    - v*(A1@c + A1f@(B1@c + B0)) \
    - (1-ec)/ec*rads
    dqdt = rads
    dcqdt = np.hstack((dcdt, dqdt))
    return dcqdt
def adssim():
    # parameters
    ec = 0.4
    ep = 0.5
    L = 3e-2
    Dax = 1e-8
    v = 0.7e-3
    cin = 0.1
    qmax = 5
    kdes = 50
    kads = 1000
    # relations
    ee = ec + (1-ec)*ep
    #mesh
    N = 20
    h = L/N
    x = np.arange(h/2, L , h) #start, stop, step
    # init
    cqinit = np.hstack(( 0*np.ones(N), 0*np.ones(N) ))
    tspan = [0, 3000]
    # solve
    sol = solve_ivp(lambda t, cq: adsmodel(t,cq,h,Dax,v,ec,ee,kads,kdes,qmax,cin),\
    tspan, cqinit, method = 'BDF')
    t = sol.t
    cq = sol.y.T
    # plot
    c = cq[:, :N]
    q = cq[:, N:]
    plt.plot(t, c)
    plt.show(block=False)

adssim()

In [ ]:
def adsmodel2(t, cq, h, Dax, v, ec, ee, kads, kdes, qmax, cin, pulseend):
    N = int(len(cq)/2)
    c = cq[:N]
    q = cq[N:]
    # puls
    if t<pulseend:
        cinp = cin
    else:
        cinp = 0
    # adsorption
    rads = kads*c*(qmax-q) - kdes*q
    # discr
    [A2,A2f] = fvm.FVMdisc2nd(N, h, '3pc')
    [A1,A1f] = fvm.FVMdisc1st(N, h, '2pb')
    [B1,B0] = fvm.FVMdiscBV(N, h, [0, 1], [[1, -cinp],[0, 0]])
    # assembly
    dcdt = Dax*(A2@c + A2f@(B1@c + B0)) \
    - v*(A1@c + A1f@(B1@c + B0)) \
    - (1-ec)/ec*rads
    dqdt = rads
    dcqdt = np.hstack((dcdt, dqdt))
    return dcqdt
def adssim2():
    # parameters
    ec = 0.4
    ep = 0.5
    L = 3e-2
    Dax = 1e-8
    v = 0.7e-3
    cin = 0.1
    qmax = 5
    kdes = 50
    kads = 1000
    pulseend = 60
    # relations
    ee = ec + (1-ec)*ep
    #mesh
    N = 20
    h = L/N
    x = np.arange(h/2, L , h) #start, stop, step
    # init
    cqinit = np.hstack(( 0*np.ones(N), 0*np.ones(N) ))
    tspan = [0, 10000]
    # solve
    sol = solve_ivp(lambda t, cq: adsmodel2(t,cq,h,Dax,v,ec,ee,kads,kdes,qmax,cin,pulse
    end),\
    tspan, cqinit, method = 'BDF')
    t = sol.t
    cq = sol.y.T
    # plot
    c = cq[:, :N]
    q = cq[:, N:]
    plt.plot(t, c[:,-1])
    plt.show(block=False)
adssim2()